In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.nn import MSELoss
from torch.nn import Module
from torch.nn import init

import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix, vstack
from scipy import sparse
from scipy.spatial.distance import cdist, cosine
from sklearn.metrics import pairwise_distances
import torch
from torch import nn as nn

from scipy.sparse import coo_matrix
import pandas as pd
import numpy as np
from numpy import diag

from torch.utils.data import DataLoader

from torch.utils.data import random_split

from torch.optim import Adam
from torch.nn import MSELoss

from os import path
from datetime import datetime

In [ ]:
import pandas as pd
import numpy as np

class DataLoader:
    def __init__(self):
        self.user_mapper = {}
        self.item_mapper = {}
        self.user_counter = 0
        self.item_counter = 0

    def load_ratings_train(self, file_path):
        df = pd.read_csv(file_path, sep=" ", header=None)
        df.columns = ['userId', 'itemId', 'rating']
        
        df, self.user_mapper, self.item_mapper, self.user_counter, self.item_counter = self._map_entities(df)
        
        return df, self.user_counter, self.item_counter

    def load_ratings_test(self, file_path):
        df = pd.read_csv(file_path, sep=" ", header=None)
        df.columns = ['userId', 'itemId', 'rating']
        
        df['userId'] = df['userId'].map(self.user_mapper)
        df['itemId'] = df['itemId'].map(self.item_mapper)
        df = df[df['userId'] < self.user_counter]
        df = df[df['itemId'] < self.item_counter]
        
        df = df.reset_index(drop=True)
        
        return df

    def load_trust(self, file_path):
        df = pd.read_csv(file_path, sep=" ", header=None)
        df.columns = ['userId1', 'userId2', 'trust']
        
        
        df['userId1'] = df['userId1'].map(self.user_mapper)
        df['userId2'] = df['userId2'].map(self.user_mapper)
        
        
        df = df[df['userId1'] < self.user_counter]
        df = df[df['userId2'] < self.user_counter]
        df = df.reset_index(drop=True)
        
        return df

    def load_rate_add(self, file_path):
        df = pd.read_csv(file_path, sep=" ", header=None)
        df.columns = ['userId', 'itemId', 'rating']        
        return df

    def _map_entities(self, df):
        for idx, row in df.iterrows():
            user_id = row['userId']
            item_id = row['itemId']
            
            if user_id not in self.user_mapper:
                self.user_mapper[user_id] = self.user_counter
                self.user_counter += 1
            if item_id not in self.item_mapper:
                self.item_mapper[item_id] = self.item_counter
                self.item_counter += 1
            
            df.at[idx, 'userId'] = self.user_mapper[user_id]
            df.at[idx, 'itemId'] = self.item_mapper[item_id]
            
            df.at[idx, 'rating'] = abs(row['rating'])
        
        return df, self.user_mapper, self.item_mapper, self.user_counter, self.item_counter


In [ ]:
# Create an instance of the DataLoader class
data_loader = DataLoader()

# Load train data
train_data, userNum, itemNum = data_loader.load_ratings_train(file_path='.../train.txt')

# Load test data
test_data = data_loader.load_ratings_test(file_path='.../test.txt')

# Load trust values between users
trust_df = data_loader.load_trust(file_path='.../trust.txt')

# Load additional rating data
rate_add_df = data_loader.load_rate_add(file_path='.../rate_add.txt')


In [ ]:
import torch
import torch.nn as nn
from torch.nn import Module
from scipy.sparse import coo_matrix
from scipy.sparse import vstack
from scipy import sparse
import numpy as np




class GNNLayer(Module):

    def __init__(self,inF,outF):

        super(GNNLayer,self).__init__()
        self.inF = inF
        self.outF = outF
        
        self.linear = torch.nn.Linear(in_features=inF,out_features=outF)
          
        self.interActTransform = torch.nn.Linear(in_features=inF,out_features=outF)
        
        self.interAttention_main = torch.nn.Linear(in_features=inF,out_features=outF)
        self.interAttention_add = torch.nn.Linear(in_features=inF,out_features=outF)
        self.interAttention_trust = torch.nn.Linear(in_features=inF,out_features=outF)

        self.a_main = nn.Parameter(torch.empty(size=(outF, 1)))
        self.a_add = nn.Parameter(torch.empty(size=(outF, 1)))
        self.a_trust = nn.Parameter(torch.empty(size=(outF, 1)))
        self.a_aug = nn.Parameter(torch.empty(size=(outF, 1)))

       

    def forward(self, laplacianMat,selfLoop,Trust_LaplacianMat, Addrate_LaplacianMat, features):
       
        L1 = laplacianMat + selfLoop
        L1 = L1.cuda()
        L2 = laplacianMat.cuda()
        

       
        
        L3 = Trust_LaplacianMat+selfLoop
        L3 = L3_1.cuda()
        L4 = Trust_LaplacianMat.cuda()
        
        L5 = Addrate_LaplacianMat+selfLoop
        L5 = L6_1.cuda()
        L6 = Addrate_LaplacianMat.cuda()
        

        
        inter_feature = torch.mul(features,features)
        x = torch.sparse.mm(L1,features)
        inter_part_main1 = self.linear(x)
        inter_part_main2 = self.interActTransform(torch.sparse.mm(L2,inter_feature)) 
        
        
        

        inter_part_main3 = torch.mm(torch.tanh(self.interAttention_main(inter_part_main1+inter_part_main2)),self.a_main )
        w_main_sum = torch.sum(inter_part_main3)/len(inter_feature)

        
        inter_part_trust = self.linear(torch.sparse.mm(L3,features))
        inter_part_trust2 = self.interActTransform(torch.sparse.mm(L4,inter_feature)) 
        
        inter_part_trust3 = torch.mm(torch.tanh(self.interAttention_trust(inter_part_trust+inter_part_trust2)),self.a_trust )
        w_trust_sum = torch.sum(inter_part_trust3)/len(inter_feature)

        
        inter_part_add = self.linear(torch.sparse.mm(L5,features))
        inter_part_add2 = self.interActTransform(torch.sparse.mm(L6,inter_feature)) 
            
        
        inter_part_add3 = torch.mm(torch.tanh(self.interAttention_add(inter_part_add+inter_part_add2)),self.a_add )
        w_add_sum = torch.sum(inter_part_add3)/len(inter_feature)
         
        W = torch.randn(1, 3)
        W[0,0] = w_main_sum
        W[0,1] = w_add_sum
        W[0,2] = w_trust_sum
        m = nn.Softmax(dim=1)
        Beta = m(W)



        return  (Beta[0,0]) * inter_part_main1 + ((Beta[0,1]) * inter_part_add) + ((Beta[0,2]) * inter_part_trust) + ((Beta[0,0]) *  inter_part_main2) + ((Beta[0,1]) *inter_part_add2) + ((Beta[0,2]) * inter_part_trust2) # imputation + trust
        
           

class GCF(Module):

    def __init__(self,userNum,itemNum,rt, trust, rt_add, embedSize=100,layers=[100,80,50],useCuda=False):
        super(GCF,self).__init__()
        
        self.useCuda = useCuda
        self.userNum = userNum
        self.itemNum = itemNum
        self.uEmbd = nn.Embedding(userNum,embedSize)
        self.iEmbd = nn.Embedding(itemNum,embedSize)
        self.GNNlayers = torch.nn.ModuleList()
        self.LaplacianMat, self.Trust_LaplacianMat, self.Addrate_LaplacianMat = self.buildLaplacianMat(rt,trust, rt_add, 1)
        
        self.leakyRelu = nn.LeakyReLU()
        self.selfLoop = self.getSparseEye(self.userNum+self.itemNum)

        self.transForm1 = nn.Linear(in_features=layers[-1]*2,out_features=64)
        self.transForm2 = nn.Linear(in_features=64,out_features=32)
        self.transForm3 = nn.Linear(in_features=32,out_features=1)

        for From,To in zip(layers[:-1],layers[1:]):
            self.GNNlayers.append(GNNLayer(From,To))

    def getSparseEye(self,num):
        i = torch.LongTensor([[k for k in range(0,num)],[j for j in range(0,num)]])
        val = torch.FloatTensor([1]*num)
        return torch.sparse.FloatTensor(i,val)

    def buildLaplacianMat(self,rt,trust, rt_add, finalEmbd):

        rt_item = rt['itemId'] + self.userNum
        uiMat = coo_matrix((rt['rating'], (rt['userId'], rt['itemId'])))
        uiMat_upperPart = coo_matrix((rt['rating'], (rt['userId'], rt_item)))
        uiMat = uiMat.transpose()
        uiMat.resize((self.itemNum, self.userNum + self.itemNum))

        A = sparse.vstack([uiMat_upperPart,uiMat])
        
        
        uiMat_upperPart_rtadd=sparse.dok_matrix((self.userNum,self.userNum + self.itemNum), dtype=np.float32) 
        uiMat_rtadd=sparse.dok_matrix((self.itemNum,self.userNum + self.itemNum), dtype=np.float32) 

        
        rtadd_item = rt_add['itemId'] + self.userNum
        uiMat_rtadd = coo_matrix((rt_add['rating'], (rt_add['userId'], rt_add['itemId'])))
        uiMat_upperPart_rtadd = coo_matrix((rt_add['rating'], (rt_add['userId'], rtadd_item)))
        uiMat_rtadd = uiMat_rtadd.transpose()
        uiMat_rtadd.resize((self.itemNum, self.userNum + self.itemNum))
        uiMat_upperPart_rtadd.resize((self.userNum, self.userNum + self.itemNum))

        A_rtadd = sparse.vstack([uiMat_upperPart_rtadd,uiMat_rtadd])

        
        
        ui_trust_tmp=sparse.dok_matrix((self.userNum,self.userNum + self.itemNum), dtype=np.float32) 
        iu_sim_tmp=sparse.dok_matrix((self.itemNum,self.userNum + self.itemNum), dtype=np.float32) 
        
        u_trust_idx = torch.LongTensor([i for i in range(self.userNum)]) 
        i_sim_idx = torch.LongTensor([i for i in range(self.itemNum)])+ self.userNum

        if self.useCuda == True:
            u_trust_idx = u_trust_idx.cuda()
            i_sim_idx = i_sim_idx.cuda()

        for i in range(len(trust)):
            if trust.iloc[i,0] < self.userNum and trust.iloc[i,1] < self.userNum:
                ui_trust_tmp[trust.iloc[i,0],trust.iloc[i,1]]=trust.iloc[i,2]
    
        A_trust = sparse.vstack([ui_trust_tmp,iu_sim_tmp])
        

        selfLoop = sparse.eye(self.userNum+self.itemNum)
        sumArr = (A>0).sum(axis=1)
        diag = list(np.array(sumArr.flatten())[0])
        diag = np.power(diag,-0.5)
        D = sparse.diags(diag)
        L = D * A * D
        L = sparse.coo_matrix(L)
        row = L.row
        col = L.col
        i = torch.LongTensor([row,col])
        data = torch.FloatTensor(L.data)
        SparseA = torch.sparse.FloatTensor(i,data)

       
        sumArr = (A_trust>0).sum(axis=1)
        diag = list( 1/np.array(sumArr.flatten())[0]) 
        diag = np.power(diag,-0.5)
        D = sparse.diags(diag)
        L = D * A_trust * D
        L = sparse.coo_matrix(L)
        row = L.row
        col = L.col
        i = torch.LongTensor([row,col])
        data = torch.FloatTensor(L.data)
        SparseA_trust = torch.sparse.FloatTensor(i,data,torch.Size(L.shape))
        print(SparseA_trust.shape)

        sumArr =  (A_rtadd>0).sum(axis=1)
        diag = list( np.array(sumArr.flatten())[0])
        diag = np.power(diag,-0.5)
        D = sparse.diags(diag)
        L = D * A_rtadd * D
        L = sparse.coo_matrix(L)
        row = L.row
        col = L.col
        i = torch.LongTensor([row,col])
        data = torch.FloatTensor(L.data)
        SparseA_rtadd = torch.sparse.FloatTensor(i,data,torch.Size(L.shape))

        return SparseA, SparseA_trust, SparseA_rtadd

    def getFeatureMat(self):
        uidx = torch.LongTensor([i for i in range(self.userNum)])
        iidx = torch.LongTensor([i for i in range(self.itemNum)])
        if self.useCuda == True:
            uidx = uidx.cuda()
            iidx = iidx.cuda()

        userEmbd = self.uEmbd(uidx)
        itemEmbd = self.iEmbd(iidx)
        
        features = torch.cat([userEmbd,itemEmbd],dim=0)
        return features

    def forward(self,userIdx,itemIdx):

        itemIdx = itemIdx + self.userNum
        userIdx = list(userIdx.cpu().data)
        itemIdx = list(itemIdx.cpu().data)
        
        features = self.getFeatureMat()
        Embeddings = features.clone()
        
        all_embeddings = [Embeddings]
        
        for gnn in self.GNNlayers:
            features = gnn(self.LaplacianMat,self.selfLoop ,self.Trust_LaplacianMat, self.Addrate_LaplacianMat, features)
            all_embeddings += [features.clone()]
            
        all_embeddings = torch.stack(all_embeddings, dim=1)
        final_embeddings = torch.mean(all_embeddings, dim=1)
        

        userEmbd = final_embeddings[userIdx]
        itemEmbd = final_embeddings[itemIdx]
        
        embd = torch.cat([userEmbd,itemEmbd],dim=1)
        
        
        
        embd = nn.ReLU()(self.transForm1(embd))
       
        embd = self.transForm2(embd)
        embd = self.transForm3(embd)
        prediction = embd.flatten()

        return prediction,userEmbd,itemEmbd,final_embeddings

In [ ]:
from torch.utils.data import Dataset

class ML1K(Dataset):

    def __init__(self,rt):
        super(Dataset,self).__init__()
        self.uId = list(rt['userId'])
        self.iId = list(rt['itemId'])
        self.rt = list(rt['rating'])

    def __len__(self):
        return len(self.uId)

    def __getitem__(self, item):
        return (self.uId[item],self.iId[item],self.rt[item])


In [ ]:

para = {
    'epoch':300,
    'lr':0.001,
    'batch_size':2048,
    
}


train = ML1K(train_data)
test = ML1K(test_data)

In [ ]:
from datetime import datetime

reg_lambda = 0.001
k = 5

dl = DataLoader(train,batch_size=para['batch_size'],shuffle=False,pin_memory=True)

model = GCF(userNum, itemNum, train_data, trust_df,rate_add_df, 80, layers=[80,80,80,])#.cuda()
optim = Adam(model.parameters(), lr=para['lr'])
lossfn = MSELoss()
testdl = DataLoader(test,batch_size=len(test),)

best_rmse = 1500
best_mae = 1500

for param in model.parameters():
    if param.dim() > 1:  # Only apply initialization to weights, not biases
        nn.init.xavier_uniform_(param)

start = datetime.now()
for i in range(para['epoch']):

    for id, batch in enumerate(dl):
        optim.zero_grad()
        prediction, userEmbd, itemEmbd, finalEmbd = model(batch[0], batch[1])
        prediction_list = list(prediction)
        loss = lossfn(batch[2].float(), prediction)

        l2_lambda = reg_lambda
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())

        loss = loss + l2_lambda * l2_norm

        loss.backward()
        optim.step()
    
    for data in testdl:
        prediction, userEmbd, itemEmbd, finalEmbd = model(data[0], data[1])

    loss = lossfn(data[2].float(), prediction)
    MAE = np.mean(np.abs(data[2].detach().numpy() - prediction.cpu().detach().numpy()))

    if np.sqrt(loss.item()) < best_rmse:
        best_rmse = np.sqrt(loss.item())
        rmse_epoch = i
    if MAE < best_mae:
        best_mae = MAE
        mae_epoch = i   

    print(' best_rmse Loss in test ', best_rmse, ' Rmse_epoch ', rmse_epoch, ' best_mae Loss in test  ', best_mae, ' mae_epoch ', mae_epoch)
    print('Epoch ', i, ' RMSE Loss in test *** ', np.sqrt(loss.item()), ' MAE Loss in test ', MAE)

end_time = datetime.now()

print('Time :',end_time-start,' Final Best RMSE ',best_rmse,' mae ',best_mae,' iteration_loss ',rmse_epoch,' iteration_mae ',mae_epoch)